## Imports

In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import ParameterGrid
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import sys
import pickle
import wandb

import torch.optim as optim


from utils.miscellaneous import read_config
from utils.miscellaneous import create_folder_structure_MLPvsGNN
from utils.miscellaneous import initalize_random_generators
from utils.wandb_logger import save_response_graphs_in_ML_tracker
from utils.normalization import *
from utils.load import *

from training.train import training
from training.test import testing
from training.models import * 

from utils.visualization import plot_R2, plot_loss
from matplotlib import pyplot as plt

### Parse configuration file + initializations


In [2]:
# read config files
cfg = read_config("config_unrolling.yaml")
# create folder for result
exp_name = cfg['exp_name']
data_folder = cfg['data_folder']
results_folder = create_folder_structure_MLPvsGNN(cfg, parent_folder='./experiments')

all_wdn_names = cfg['network']
initalize_random_generators(cfg, count=0)

# initialize pytorch device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
#torch.set_num_threads(12)

Creating folder: ./experiments/unrolling_WDN0401
cpu


In [3]:
# TO DO: at the moment I am not using the parsed values for batch size and num_epochs ;
# I am not using alpha as well because the loss has no "smoothness" penalty (yet)
batch_size = cfg['trainParams']['batch_size']
num_epochs = cfg['trainParams']['num_epochs']
res_columns = ['train_loss', 'valid_loss', 'test_loss', 'max_train_loss', 'max_valid_loss', 'max_test_loss',
               'min_train_loss', 'min_valid_loss', 'min_test_loss', 'r2_train', 'r2_valid',
               'r2_test', 'total_params', 'total_time', 'test_time']

# Functions

## Models
I will be Creating different models as follows:

* A simple LSTM
* An unrolled version of Heads and Flows, without static variables
* An unrolled version with Heads, Flows and static variables


## Running experiments

In [4]:
for ix_wdn, wdn in enumerate(all_wdn_names):
    print(f'\nWorking with {wdn}, network {ix_wdn + 1} of {len(all_wdn_names)}')

    # retrieve wntr data
    tra_database, val_database, tst_database = load_raw_dataset(wdn, data_folder)
    # reduce training data
    # tra_database = tra_database[:int(len(tra_database)*cfg['tra_prc'])]
    if cfg['tra_num'] < len(tra_database):
        tra_database = tra_database[:cfg['tra_num']]


    # get GRAPH datasets     
    # later on we should change this and use normal scalers from scikit (something is off here)
    tra_dataset, A12_bar = create_dataset(tra_database)
    # number of nodes
    junctions = (tra_database[0].node_type == JUNCTION_TYPE).numpy().sum()
    tanks = (tra_database[0].node_type == TANK_TYPE).numpy().sum()
    output_nodes = len(tra_dataset[0].y[0]) # remove reservoirs
    gn = GraphNormalizer(junctions + tanks, output=['pressure', 'pump_flow'])
    gn = gn.fit(tra_dataset)
    # The normalization messed with the 1H_type since we want unique IDs
    tra_dataset, _ = create_dataset(tra_database, normalizer=gn)
    val_dataset, _ = create_dataset(val_database, normalizer=gn)
    tst_dataset, _ = create_dataset(tst_database, normalizer=gn)
    node_size, edge_size = tra_dataset[0].x.size(-1), tra_dataset[0].edge_attr.size(-1)
    
    # dataloader
    # transform dataset for MLP
    # We begin with the MLP versions, when I want to add GNNs, check Riccardo's code
    A10, A12 = create_incidence_matrices(tra_dataset, A12_bar)
    tra_dataset_MLP, num_inputs, indices = create_dataset_MLP_from_graphs(tra_dataset)
    val_dataset_MLP = create_dataset_MLP_from_graphs(val_dataset)[0]
    tst_dataset_MLP = create_dataset_MLP_from_graphs(tst_dataset)[0]
    tra_loader = torch.utils.data.DataLoader(tra_dataset_MLP,
                                             batch_size=batch_size, shuffle=True, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(val_dataset_MLP,
                                             batch_size=batch_size, shuffle=False, pin_memory=True)
    tst_loader = torch.utils.data.DataLoader(tst_dataset_MLP,
                                             batch_size=batch_size, shuffle=False, pin_memory=True)
    # loop through different algorithms
    for algorithm in cfg['algorithms']:
        # Importing of configuration parameters
        hyperParams = cfg['hyperParams'][algorithm]
        all_combinations = ParameterGrid(hyperParams)

        # create results dataframe
        results_df = pd.DataFrame(list(all_combinations))
        results_df = pd.concat([results_df,
                                pd.DataFrame(index=np.arange(len(all_combinations)),
                                             columns=list(res_columns))], axis=1)

        for i, combination in enumerate(all_combinations):
            # wandb.init(project="unrolling-epanet", entity="mertz")
            print(f'{algorithm}: training combination {i + 1} of {len(all_combinations)}\n')
            
            combination['indices'] = indices
            combination['junctions'] = junctions
            combination['num_outputs'] = output_nodes

            # model creation
            model = getattr(sys.modules[__name__], algorithm)(**combination).float().to(device)

            # get combination dictionary to determine how are indices made
            total_parameters = sum(p.numel() for p in model.parameters())
            print("Total number of parameters is", total_parameters)

            # model optimizer
            optimizer = optim.Adam(params=model.parameters(), betas=(0.9, 0.999), **cfg['adamParams'])

            # training
            patience = cfg['earlyStopping']['patience']
            lr_rate = cfg['earlyStopping']['divisor']
            lr_epoch = cfg['earlyStopping']['epoch_frequency']
            train_config = {"Patience": patience, "Learning Rate Divisor": lr_rate, "LR Epoch Division": lr_epoch}
            model, tra_losses, val_losses, elapsed_time = training(model, optimizer, tra_loader, val_loader,
                                                                   patience=patience, report_freq=0,
                                                                   n_epochs=num_epochs,
                                                                   alpha=0, lr_rate=lr_rate, lr_epoch=lr_epoch,
                                                                   normalization=None, path="experiments")
            
            loss_plot = plot_loss(tra_losses, val_losses, f'{results_folder}/{wdn}/{algorithm}/loss/{i}')
            R2_plot = plot_R2(model, val_loader, f'{results_folder}/{wdn}/{algorithm}/R2/{i}', normalization=gn)[1]
            # store training history and model
            pd.DataFrame(data=np.array([tra_losses, val_losses]).T).to_csv(
                f'{results_folder}/{wdn}/{algorithm}/hist/{i}.csv')
            torch.save(model, f'{results_folder}/{wdn}/{algorithm}/models/{i}.csv')

            # compute and store predictions, compute r2 scores
            losses = {}
            max_losses = {}
            min_losses = {}
            r2_scores = {}
            for split, loader in zip(['training', 'validation', 'testing'], [tra_loader, val_loader, tst_loader]):
                losses[split], max_losses[split], min_losses[split], pred, real, test_time = testing(model, loader, normalization=gn)
                r2_scores[split] = r2_score(real, pred)
                if i == 0:
                    pd.DataFrame(data=real.reshape(-1, output_nodes)).to_csv(
                        f'{results_folder}/{wdn}/{algorithm}/pred/{split}/real.csv')  # save real obs
                pd.DataFrame(data=pred.reshape(-1, output_nodes)).to_csv(
                    f'{results_folder}/{wdn}/{algorithm}/pred/{split}/{i}.csv')

            # log_wandb_data(combination, wdn, algorithm, len(tra_database), len(val_database), len(tst_database), cfg, train_config, loss_plot, R2_plot)
            # store results
            results_df.loc[i, res_columns] = (losses['training'], losses['validation'], losses['testing'],
                                              max_losses['training'], max_losses['validation'], max_losses['testing'],
                                              min_losses['training'], min_losses['validation'], min_losses['testing'],
                                              r2_scores['training'], r2_scores['validation'], r2_scores['testing'],
                                              total_parameters, elapsed_time, test_time)
            
        # Calculate dummy model    
        # wandb.finish()
        # save graph normalizer
        # with open(f'{results_folder}/{wdn}/{algorithm}/gn.pickle', 'wb') as handle:
        #     pickle.dump(gn, handle, protocol=pickle.HIGHEST_PROTOCOL)
        # 
        with open(f'{results_folder}/{wdn}/{algorithm}/model.pickle', 'wb') as handle:
            torch.save(model, handle)
        results_df.to_csv(f'{results_folder}/{wdn}/{algorithm}/results_{algorithm}.csv')
        
        print('Folder where model was saved is', results_folder)



Working with FOS_pump_4, network 1 of 1
UnrollingModel: training combination 1 of 1

Total number of parameters is 22809


 10%|█         | 50/500 [00:12<02:05,  3.58it/s]

Learning rate is divided by 2 to: 0.005


 20%|██        | 100/500 [00:25<01:43,  3.85it/s]

Learning rate is divided by 2 to: 0.0025


 30%|███       | 150/500 [00:40<01:30,  3.88it/s]

Learning rate is divided by 2 to: 0.00125


 40%|███▉      | 198/500 [00:53<01:22,  3.68it/s]

Early Stopping


Folder where model was saved is ./experiments/unrolling_WDN0401


In [1]:
from utils.Dashboard import Dashboard
_, _, _, pred, real, timed = testing(model, tst_loader, normalization=gn)

pred = gn.denormalize_multiple(pred, output_nodes)
real = gn.denormalize_multiple(real, output_nodes)

dummy = Dummy(junctions + tanks).evaluate(real)
# Array below is created to ensure proper indexing of the nodes when displaying
type_array = (tst_database[0].node_type == 0) | (tst_database[0].node_type == 2)
d = Dashboard(pd.DataFrame(real[0:24, :]), pd.DataFrame(pred[0:24, :]),
              to_networkx(tst_dataset[0], node_attrs=['pos', 'ID']), type_array)
f = d.display_results()

# for i in range(0, len(real[:]), 5):
# for i in [0, 1, 6, 26, 36, 37]:
#     plt.plot(real[0:100, i], label="Real")
#     plt.plot(pred[0:100, i], label="Predicted")
#     plt.plot(dummy[0:100, i], label="Dummy")
    
#     plt.ylabel('Head')
#     plt.xlabel('Timestep')
    
#     plt.legend()
#     names = {0: 'Next to Reservoir', 1: 'Random Node', 6: 'Next to Tank', 26: 'Random Node', 36: 'Tank', 37: 'Pump'}
#     plt.title(names[i])
#     # save_response_graphs_in_ML_tracker(real, pred, names[i], i)
#     plt.show()
#     plt.close()

# plt.plot(real[0:100, 37], label="Real")
# plt.plot(pred[0:100, 37], label="Predicted")
# plt.plot(dummy[0:100, 37], label="Dummy")
# plt.ylabel('LPS')
# plt.xlabel('Timestep')
# 
# plt.legend()
# plt.title(names[37])
# plt.show()
# plt.close()
# save_response_graphs_in_ML_tracker(real, pred, names[i], i)
# Create a table

# Add Plotly figure as HTML file into Table
table = wandb.Table(columns = ["Figure" + str(i)])
# with open('./my_HTML_' + str(i) + '.html', 'r', encoding='utf-8') as file:
#     html_content = file.read()
# table.add_data(wandb.Html(html_content))
display(f)
# wandb.finish()

NameError: name 'testing' is not defined

In [6]:
gn.transform_array([1,2,3,2,1,0], 'pressure')
gn.inverse_transform_array(gn.transform_array([1,2,3,2,1,0], 'pressure'), 'pressure')

C:\Uni\Thesis\Albert\GGNet\utils\normalization.py:159: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



tensor([ 1.0000e+00,  2.0000e+00,  3.0000e+00,  2.0000e+00,  1.0000e+00,
        -7.1054e-15], dtype=torch.float64)

In [ ]:

def nse(observed, simulated):
    """
    Calculate Nash-Sutcliffe Efficiency (NSE) for 2D tensors.

    Args:
    observed (torch.Tensor): Tensor containing observed values.
    simulated (torch.Tensor): Tensor containing simulated values.

    Returns:
    NSE (float): Nash-Sutcliffe Efficiency value.
    """
    assert observed.shape == simulated.shape, "Input tensors must have the same shape."
    
    numerator = torch.sum((observed - simulated) ** 2)
    denominator = torch.sum((observed - torch.mean(observed)) ** 2)
    
    nse = 1 - (numerator / denominator)
    return nse.item()

dummy_score = r2_score(real, dummy, multioutput='variance_weighted')
model_score = r2_score(real, pred, multioutput='variance_weighted')
print("R2-values \n", "Dummy:", dummy_score, "\n Model", model_score)

dummy_score = mean_absolute_error(real, dummy)
model_score = mean_absolute_error(real, pred)
print("MAE-values \n", "Dummy:", dummy_score, "\n Model", model_score)

dummy_score = mean_squared_error(real, dummy)
model_score = mean_squared_error(real, pred)
print("MSE-values \n", "Dummy:", dummy_score, "\n Model", model_score)


dummy_score = mean_squared_error(real, dummy, squared=False)
model_score = mean_squared_error(real, pred, squared=False)
print("RMSE-values \n", "Dummy:", dummy_score, "\n Model", model_score)

dummy_score = nse(real, dummy)
model_score = nse(real, pred)
print("NSE-values \n", "Dummy:", dummy_score, "\n Model", model_score)

dummy_scores = []
model_scores = []

for i in range(38):
    dummy_score = nse(real[:, i], dummy[:, i])
    model_score = nse(real[:, i], pred[:, i])
    dummy_scores.append(dummy_score)
    model_scores.append(model_score)
    
    print("NSE-values", i, dummy_score, model_score)


In [8]:
# plt.plot(dummy_scores, label="Dummy", color='green')
# plt.xlabel('Node')
# plt.ylabel('NSE')
# plt.title('Dummy')
# plt.show()
# plt.close()
# plt.plot(model_scores, label="Model", color='orange')
# plt.xlabel('Node')
# plt.ylabel('NSE')
# plt.title('Model (UM)')
# plt.show()
# plt.close()

# Create a colormap
cmap = plt.get_cmap('RdYlGn')

switch_7 = dummy_scores[6]
dummy_scores[6] = dummy_scores[16]
dummy_scores[16] = switch_7
# Get the rgba values from the colormap
dummy_colors = cmap(dummy_scores)


switch_7 = model_scores[6]
model_scores[6] = model_scores[16]
model_scores[16] = switch_7
model_colors = cmap(model_scores)

# Print the numbers with their corresponding colors

zip_dummy = zip(dummy_scores, dummy_colors)
zip_model = zip(model_scores, model_colors)

index = 0
for num, color in zip_dummy:
    index += 1
    r, g, b, _ = color
    print(index, f"\033[38;2;{int(r * 255)};{int(g * 255)};{int(b * 255)}m{format(num, '.2f')}\033[0m")

print('\n')
# Print the numbers with their corresponding colors
index = 0
for num, color in zip_model:
    index += 1
    r, g, b, _ = color
    print(index, f"\033[38;2;{int(r * 255)};{int(g * 255)};{int(b * 255)}m{format(num, '.2f')}\033[0m")
    


1 0.97
2 0.96
3 0.89
4 0.75
5 0.50
6 0.83
7 -29.81
8 0.66
9 0.81
10 0.97
11 0.93
12 0.77
13 0.64
14 0.78
15 0.92
16 0.99
17 0.16
18 0.97
19 0.81
20 0.82
21 0.83
22 0.97
23 0.99
24 0.32
25 0.99
26 0.90
27 0.96
28 0.81
29 0.97
30 0.94
31 0.98
32 0.97
33 0.99
34 0.99
35 0.92
36 0.88
37 -90.26
38 0.00


1 0.72
2 0.67
3 0.67
4 0.70
5 0.55
6 0.64
7 0.76
8 0.65
9 0.68
10 0.72
11 0.72
12 0.71
13 0.63
14 0.70
15 0.72
16 0.72
17 0.67
18 0.72
19 0.71
20 0.71
21 0.70
22 0.72
23 0.71
24 0.66
25 0.71
26 0.72
27 0.72
28 0.64
29 0.68
30 0.65
31 0.72
32 0.72
33 0.72
34 0.72
35 0.69
36 0.71
37 0.75
38 0.28


In [9]:
import time
from datetime import datetime


with open(f'{results_folder}/{wdn}/{algorithm}/model.pickle', 'rb') as handle:
    loaded_model = torch.load(handle)
    loaded_model.eval()
    

# print(len(tra_dataset_MLP), len(tra_dataset_MLP[0]), len(tra_dataset_MLP[0][0]))
input = tra_dataset_MLP[0][0].unsqueeze(0).to(device)
print(input.shape)

start_time = time.time()
print(start_time)
print('Tra loader length', len(tra_loader))
for batch in tra_loader:
    print(len(batch), len(batch[0]), len(batch[0][0])   )
    input = batch[0].to(device)
    output = model(input)

end_time = time.time()
print(end_time)

print(f"Simulation time: {end_time - start_time}")

output = output.detach().cpu().numpy()
real = gn.inverse_transform_array(output, 'pressure')
print(real)

torch.Size([1, 950])
1706783825.4427874
Tra loader length 7
2 128 950
2 128 950
2 128 950
2 128 950
2 128 950
2 128 950
2 32 950
1706783825.4525552
Simulation time: 0.009767770767211914
tensor([27.8412, 28.3808, 29.4993,  ..., 26.9995, 19.9920, 31.9231])


In [10]:
import os
import regex as re

# Directory path where you want to search
directory_path = "./experiments"

# Get a list of all subdirectories in the specified directory
subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]

# Filter and extract the numbers from directory names
wdn_numbers = []
for subdir in subdirectories:
    match = re.match(r'unrolling_WDN(\d{4})', subdir)
    if match:
        wdn_numbers.append(int(match.group(1)))

# Find the latest WDN number
latest_wdn_number = None
if wdn_numbers:
    latest_wdn_number = max(wdn_numbers)
    latest_wdn_folder = f'unrolling_WDN{latest_wdn_number:04d}'
    print(f"The latest WDN folder is: {latest_wdn_folder}")
else:
    print("No WDN folders found in the specified directory.")

if latest_wdn_folder is not None:
    real = pd.read_csv(f'./experiments/unrolling_WDN{latest_wdn_number:04d}/FOS_tank/LSTM/pred/testing/real.csv').drop(
        columns=['Unnamed: 0'])
    lstm_pred = pd.read_csv(
        f'./experiments/unrolling_WDN{latest_wdn_number:04d}/FOS_tank/LSTM/pred/testing/0.csv').drop(
        columns=['Unnamed: 0'])
    unrolling_pred = pd.read_csv(
        f'./experiments/unrolling_WDN{latest_wdn_number:04d}/FOS_tank/BaselineUnrolling/pred/testing/0.csv').drop(
        columns=['Unnamed: 0'])

The latest WDN folder is: unrolling_WDN0401


FileNotFoundError: [Errno 2] No such file or directory: './experiments/unrolling_WDN0401/FOS_tank/LSTM/pred/testing/real.csv'

In [ ]:
import matplotlib.pyplot as plt

# Not sure if below makes sense since we now have an extra dimension
res = real.sub(lstm_pred).pow(2).sum(axis=0)
tot = real.sub(lstm_pred.mean(axis=0)).pow(2).sum(axis=0)
r2_lstm = 1 - res / tot
res = real.sub(unrolling_pred).pow(2).sum(axis=0)
tot = real.sub(unrolling_pred.mean(axis=0)).pow(2).sum(axis=0)
r2_unrolling = 1 - res / tot
r2s = pd.concat([r2_lstm, r2_unrolling], axis=1).rename(columns={0: 'LSTM', 1: 'Base-U'})
fig, ax = plt.subplots()
r2s.plot.box(ax=ax)
ax.set_title("$R^2$ Scores Comparison for PES")
ax.set_ylabel('$R^2$ Score')
plt.show()

In [ ]:
model = torch.load(f'{results_folder}/{wdn}/{algorithm}/model.pickle')